<div class="alert alert-block alert-info">
    <p><b>Курс: </b>Big Data for Data Science</p>
    <p><b>Раздел: </b>3. Apache Spark - задачи</p>
</div>

In [18]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки
<br>|-- payment_type = тип оплаты

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [3]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [4]:
spark

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [6]:
#Ваш код загрузки
df = spark.read.parquet('_data/taxi_data.parquet')

№1 Посчитайте количество загруженных строк.

In [24]:
# Число строк
df.count()

2540712

In [9]:
df.show()

+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare| tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75| 2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0|  3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0|  0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25|  2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0|  0.0|       5.0|        Cash|
|   4666| 2016-12-30 18:00:00|2016-12-30 18:45:00|        2400|      18.2|46.75|10.15|      61.4| Credit Card|
|

Посмотрим схему данных:

In [10]:
df.printSchema('trip_miles', )

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.corr">corr</a> & <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cov">cov</a>

In [26]:
# Ваш код
round(df.corr('trip_miles', 'trip_total'), 5)

0.44816

In [27]:
# Ваш код
round(df.cov('trip_miles', 'trip_total'), 5)

71.96914

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe">describe</a>

In [29]:
# Ваш код
df['trip_miles', 'trip_total'].describe().show()

+-------+------------------+------------------+
|summary|        trip_miles|        trip_total|
+-------+------------------+------------------+
|  count|           2540677|           2540672|
|   mean|3.0005873828090266|15.913560215564042|
| stddev|  5.25716922943536|30.546699217618237|
|    min|               0.0|               0.0|
|    max|             900.0|           9276.69|
+-------+------------------+------------------+



№4 Найдите самый НЕ популярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy">groupBy</a> <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [58]:
# Ваш код
payment_types_groupped = df.groupBy('payment_type').count().sort('count', ascending=True).collect()
payment_types_groupped

[Row(payment_type='Way2ride', count=3),
 Row(payment_type='Pcard', count=878),
 Row(payment_type='Prcard', count=968),
 Row(payment_type='Dispute', count=1842),
 Row(payment_type='Unknown', count=5180),
 Row(payment_type='No Charge', count=12843),
 Row(payment_type='Credit Card', count=1108843),
 Row(payment_type='Cash', count=1410155)]

In [56]:
payment_types_groupped[0][0]

'Way2ride'

№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [112]:
# Ваш код
df.groupBy('taxi_id').count().sort('count', ascending=False).collect()[0][0]

316

№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where">where</a>

In [74]:
# Ваш код
cash = df.where(df['payment_type'] == 'Cash').describe()

In [110]:
cash.show()

+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+------------+
|summary|          taxi_id|      trip_seconds|        trip_miles|              fare|                tips|        trip_total|payment_type|
+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+------------+
|  count|          1409697|           1409967|           1410122|           1410119|             1410119|           1410119|     1410155|
|   mean|4360.783044157716| 712.4986067049796|2.2804813271493494|11.194248598877067|0.003325194540318936|12.035261470840307|        null|
| stddev|2516.059908686248|1342.1036613278843| 4.590882504814429| 27.17314538539178|  0.2083038815193268| 36.73317411286132|        null|
|    min|                0|                 0|               0.0|               0.0|                 0.0|               0.0|        Cash|
|    max|             8760|       

№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [125]:
########################################################################### Ваш код
df.groupBy('taxi_id', 'trip_miles').count()[df['trip_miles'] > 100].count()

106

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [ ]:
# Ваш код

№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.

In [ ]:
# Ваш код

№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hour">hour</a>

In [19]:
from pyspark.sql.functions import hour

In [20]:
# Ваш код

№11 Сколько поездок началось во второй четверти дня?

In [21]:
# Ваш код

№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast">cast</a>

In [ ]:
from pyspark.sql.types import DateType

In [ ]:
# Ваш код

№13 Сколько было заказов в дату с наибольшим спросом?

In [ ]:
# Ваш код

Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [ ]:
df_car = # Ваш код загрузки

In [ ]:
df_car.show()

№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split">split</a>

In [23]:
from pyspark.sql.functions import split

In [22]:
# Ваш код

№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join</a>

In [ ]:
# Ваш код

Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [12]:
# Ваш код с coalesce(1)

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [13]:
# Ваш код с printSchema() для DataFrame из csv

Не забудьте посетить SparkUI и изучить историю ваших задач.

In [ ]:
spark